In [1]:
# Main file funcitons
from configs import *

# Main manipulation functions
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import  Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import  cross_val_score, cross_val_predict
from sklearn.metrics import  confusion_matrix, precision_score, recall_score, f1_score
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
%matplotlib inline

In [2]:
adult_train_data = pd.read_csv("dataset/analysis_adult.csv")
adult_train_data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,Occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income_average
0,5.617541,Self-emp-not-inc,273.932785,Bachelors,21.242733,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,5.086120,Private,409.397625,HS-grad,12.826953,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,5.733697,Private,424.301690,11th,9.013060,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,4.524024,Private,495.076382,Bachelors,21.242733,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,5.035827,Private,460.209461,Masters,23.487457,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


In [3]:
le = preprocessing.LabelEncoder()
income_average = le.fit_transform(adult_train_data['income_average'])
income_average[:10]

array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1])

In [4]:
adult_train_data['income_average'] = income_average

In [5]:
adult_train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32530 entries, 0 to 32529
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             32500 non-null  float64
 1   workclass       32530 non-null  object 
 2   fnlwgt          32500 non-null  float64
 3   education       32530 non-null  object 
 4   education-num   32500 non-null  float64
 5   marital-status  32530 non-null  object 
 6   Occupation      32530 non-null  object 
 7   relationship    32530 non-null  object 
 8   race            32530 non-null  object 
 9   sex             32530 non-null  object 
 10  capital-gain    32530 non-null  int64  
 11  capital-loss    32530 non-null  int64  
 12  hours-per-week  32530 non-null  int64  
 13  native-country  32530 non-null  object 
 14  income_average  32530 non-null  int64  
dtypes: float64(3), int64(4), object(8)
memory usage: 3.7+ MB


In [6]:
adult_train_data = adult_train_data.sample(frac = 1)
  
adult_train_data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,Occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income_average
10820,4.931782,Private,387.772728,HS-grad,28.125787,Never-married,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
4727,4.100884,Private,428.155097,HS-grad,16.915466,Divorced,Craft-repair,Other-relative,White,Male,0,0,40,United-States,0
20181,4.100884,Private,316.927165,HS-grad,21.242733,Never-married,Sales,Not-in-family,White,Male,0,1504,37,United-States,0
15991,5.453373,Private,390.483585,Bachelors,21.242733,Divorced,Exec-managerial,Unmarried,White,Female,0,0,40,United-States,0
4406,3.938517,Private,391.027607,HS-grad,14.839411,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,28,United-States,0


## Split Trainging Data

We split data to train and validation based on Survived features to ensure we have representative number of classes in each part of that split.

In [7]:
adult_train_data.isna().sum()

age               30
workclass          0
fnlwgt            30
education          0
education-num     30
marital-status     0
Occupation         0
relationship       0
race               0
sex                0
capital-gain       0
capital-loss       0
hours-per-week     0
native-country     0
income_average     0
dtype: int64

In [8]:
adult_train_data['age'] = adult_train_data.fillna(adult_train_data['age'].median())
adult_train_data['fnlwgt'] = adult_train_data.fillna(adult_train_data['fnlwgt'].median())
adult_train_data['education-num'] = adult_train_data.fillna(adult_train_data['education-num'].median())
adult_train_data.isna().sum()

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
Occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
income_average    0
dtype: int64

In [9]:
train_data, test_data = split_data("income_average", adult_train_data)
train_data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,Occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income_average
11943,5.695595,Private,5.695595,HS-grad,5.695595,Divorced,Craft-repair,Not-in-family,White,Male,0,0,40,United-States,0
29979,4.251497,State-gov,4.251497,Masters,4.251497,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,1887,45,United-States,1
23521,5.808155,State-gov,5.808155,Some-college,5.808155,Divorced,Adm-clerical,Unmarried,Black,Male,0,0,40,United-States,0
20755,4.251497,Private,4.251497,Some-college,4.251497,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,6,United-States,0
15719,5.410496,Local-gov,5.410496,Masters,5.410496,Never-married,Prof-specialty,Not-in-family,White,Female,0,0,40,United-States,0


In [10]:
test_data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,Occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income_average
4217,3.762153,Private,3.762153,Some-college,3.762153,Divorced,Sales,Unmarried,White,Female,0,0,36,United-States,0
25289,4.931782,Private,4.931782,Prof-school,4.931782,Never-married,Prof-specialty,Unmarried,White,Male,27828,0,45,Germany,1
9643,5.495487,Private,5.495487,HS-grad,5.495487,Married-civ-spouse,Exec-managerial,Husband,White,Male,7298,0,40,United-States,1
10614,5.536868,Private,5.536868,10th,5.536868,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,0
18718,3.667827,Local-gov,3.667827,HS-grad,3.667827,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,1


In [11]:
check_split_error(adult_train_data, train_data, test_data, 'income_average')

,overall,train,validation,train_error,test_error
0,0.241008,0.241008,0.241008,0.0,0.0
1,0.758992,0.758992,0.758992,0.0,0.0


In [12]:
train_data.isna().sum()

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
Occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
income_average    0
dtype: int64

In [13]:
test_data.isna().sum()

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
Occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
income_average    0
dtype: int64

In [14]:
y_train   = np.array(train_data['income_average'])
train_data.drop('income_average', axis=1, inplace=True)

y_test   = np.array(test_data['income_average'])
test_data.drop('income_average', axis=1, inplace=True)

print(y_train.shape)
print(y_test.shape)

(29277,)
(3253,)


/home/abdelrahman/.local/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [15]:
numerical_attr    = [ 'age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']

categorical_attr = ['workclass', 'education', 'marital-status', 'Occupation', 'relationship', 'race',
                   'sex', 'native-country']

full_pipeline = ColumnTransformer([
    ('cat_pipeline', OneHotEncoder(sparse=False), categorical_attr),
('num_pipeline', MinMaxScaler(), numerical_attr),

])

X_train = full_pipeline.fit_transform(train_data)

# Naive Bayes Classifier

In [16]:
def predict_result(model, X,y):
    predicted = model.predict(X)
    print("Accuracy", round(f1_score(y, predicted, average='micro')*100, 2))
    print("="*50)
    print(confusion_matrix(y, predicted))
    return True

In [17]:
clf_model_1 = GaussianNB().fit(X_train, y_train)
print("================  Accuracy ======================")
predict_result(clf_model_1, X_train, y_train)

================  Accuracy ======================
Accuracy 58.65
[[10540 11681]
 [  425  6631]]


True

In [18]:
clf_model_2 = LogisticRegression().fit(X_train, y_train)
print("================  Accuracy ======================")
predict_result(clf_model_2, X_train, y_train)

================  Accuracy ======================
Accuracy 84.81
[[20721  1500]
 [ 2946  4110]]


/home/abdelrahman/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


True